In [2]:
%pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sidsriram62067/data-set-for-upi-fraud")

print("Path to dataset files:", path)

/opt/miniconda3/envs/tf_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 24.4M/24.4M [00:04<00:00, 5.78MB/s]

Extracting files...


Path to dataset files: /Users/abhishekmarutikarennavar/.cache/kagglehub/datasets/sidsriram62067/data-set-for-upi-fraud/versions/1
